##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Redes

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/8_networks_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

En este colab cubriremos cómo definir redes personalizadas para los agentes. Las redes nos ayudan a definir el modelo que entrenan los agentes. En TF-Agents hay varios tipos de redes que son útiles para los agentes:

**Principales redes**

- **QNetwork**: esta red, usada en Qlearning para entornos con acciones discretas, asigna una observación a estimaciones de valor para cada acción posible.
- **CriticNetworks**: también conocidas como `ValueNetworks` en la literatura, aprenden a estimar alguna versión de una función de Valor que asigna algún estado a una estimación del rendimiento esperado de una política. Estas redes estiman qué tan bueno es el estado en el que se encuentra actualmente el agente.
- **ActorNetworks**: aprenden una relación entre observaciones y acciones. Nuestras políticas suelen utilizar estas redes para generar acciones.
- **ActorDistributionNetworks**: similares a `ActorNetworks`, pero generan una distribución a partir de la cual una política puede extraer muestras para generar acciones.

**Redes ayudantes**

- **EncodingNetwork**: permite a los usuarios definir fácilmente una asignación de capas de preprocesamiento para aplicarlas a la entrada de una red.
- **DynamicUnrollLayer**: restablece automáticamente el estado de la red en los límites del episodio a medida que se aplica durante una secuencia de tiempo.
- **ProjectionNetwork**: redes similares a `CategoricalProjectionNetwork` o a `NormalProjectionNetwork` toman entradas y generan los parámetros necesarios para generar distribuciones normales o categóricas.

Todos los ejemplos en TF-Agents vienen con redes previamente configuradas. No obstante, estas redes no están configuradas para gestionar observaciones complejas.

Si tiene un entorno que expone más de una observación o acción, y necesita personalizar sus redes, ¡este tutorial es para usted!

## Preparación

Si todavía no ha instalado tf-agents, ejecute los siguientes comandos:

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import random_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.networks import encoding_network
from tf_agents.networks import network
from tf_agents.networks import utils
from tf_agents.specs import array_spec
from tf_agents.utils import common as common_utils
from tf_agents.utils import nest_utils

## Cómo definir las redes

### API de red

En TF-Agents, subclasificamos a partir de [redes](https://github.com/tensorflow/agents/blob/master/tf_agents/networks/network.py) de Keras. Esto nos permite lo siguiente:

- Simplificar operaciones de copia necesarias al crear redes objetivo.
- Crear variables automáticamente al llamar `network.variables()`.
- Validar entradas en función de las input_specs de la red.

##EncodingNetwork Como mencionamos anteriormente, `EncodingNetwork` nos permite definir fácilmente una asignación de capas de preprocesamiento que se aplicarán a la entrada de una red para generar alguna codificación.

EncodingNetwork se compone de las siguientes capas, mayormente opcionales:

- Capas de preprocesamiento
- Combinador de preprocesamiento
- Conv2D
- Flatten
- Dense

Lo especial de las redes de codificación es que se aplica el preprocesamiento de entrada. El preprocesamiento de entrada se consigue con las capas `preprocessing_layers` y `preprocessing_combiner`.  Cada una de ellas puede especificarse como una estructura anidada. Si el nido `preprocessing_layers` es menos profundo que `input_tensor_spec`, entonces las capas incluirán subnidos. Por ejemplo, si

```
input_tensor_spec = ([TensorSpec(3)] * 2, [TensorSpec(3)] * 5)
preprocessing_layers = (Layer1(), Layer2())
```

entonces, el preprocesamiento llamará lo siguiente:

```
preprocessed = [preprocessing_layers[0](observations[0]),
                preprocessing_layers[1](observations[1])]
```

No obstante, si

```
preprocessing_layers = ([Layer1() for _ in range(2)],
                        [Layer2() for _ in range(5)])
```

entonces, el preprocesamiento llamará lo siguiente:

```python
preprocessed = [
  layer(obs) for layer, obs in zip(flatten(preprocessing_layers),
                                    flatten(observations))
]
```


### Redes personalizadas

Para crear sus propias redes, solo debe anular los métodos `__init__` y `call`. Creemos una red personalizada poniendo en práctica lo que aprendimos sobre `EncodingNetworks` para crear un ActorNetwork que tome observaciones que contengan una imagen y un vector.


In [ ]:
class ActorNetwork(network.Network):

  def __init__(self,
               observation_spec,
               action_spec,
               preprocessing_layers=None,
               preprocessing_combiner=None,
               conv_layer_params=None,
               fc_layer_params=(75, 40),
               dropout_layer_params=None,
               activation_fn=tf.keras.activations.relu,
               enable_last_layer_zero_initializer=False,
               name='ActorNetwork'):
    super(ActorNetwork, self).__init__(
        input_tensor_spec=observation_spec, state_spec=(), name=name)

    # For simplicity we will only support a single action float output.
    self._action_spec = action_spec
    flat_action_spec = tf.nest.flatten(action_spec)
    if len(flat_action_spec) > 1:
      raise ValueError('Only a single action is supported by this network')
    self._single_action_spec = flat_action_spec[0]
    if self._single_action_spec.dtype not in [tf.float32, tf.float64]:
      raise ValueError('Only float actions are supported by this network.')

    kernel_initializer = tf.keras.initializers.VarianceScaling(
        scale=1. / 3., mode='fan_in', distribution='uniform')
    self._encoder = encoding_network.EncodingNetwork(
        observation_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
        conv_layer_params=conv_layer_params,
        fc_layer_params=fc_layer_params,
        dropout_layer_params=dropout_layer_params,
        activation_fn=activation_fn,
        kernel_initializer=kernel_initializer,
        batch_squash=False)

    initializer = tf.keras.initializers.RandomUniform(
        minval=-0.003, maxval=0.003)

    self._action_projection_layer = tf.keras.layers.Dense(
        flat_action_spec[0].shape.num_elements(),
        activation=tf.keras.activations.tanh,
        kernel_initializer=initializer,
        name='action')

  def call(self, observations, step_type=(), network_state=()):
    outer_rank = nest_utils.get_outer_rank(observations, self.input_tensor_spec)
    # We use batch_squash here in case the observations have a time sequence
    # compoment.
    batch_squash = utils.BatchSquash(outer_rank)
    observations = tf.nest.map_structure(batch_squash.flatten, observations)

    state, network_state = self._encoder(
        observations, step_type=step_type, network_state=network_state)
    actions = self._action_projection_layer(state)
    actions = common_utils.scale_to_spec(actions, self._single_action_spec)
    actions = batch_squash.unflatten(actions)
    return tf.nest.pack_sequence_as(self._action_spec, [actions]), network_state

Creemos un `RandomPyEnvironment` para generar observaciones estructuradas y validar nuestra implementación.

In [ ]:
action_spec = array_spec.BoundedArraySpec((3,), np.float32, minimum=0, maximum=10)
observation_spec =  {
    'image': array_spec.BoundedArraySpec((16, 16, 3), np.float32, minimum=0,
                                        maximum=255),
    'vector': array_spec.BoundedArraySpec((5,), np.float32, minimum=-100,
                                          maximum=100)}

random_env = random_py_environment.RandomPyEnvironment(observation_spec, action_spec=action_spec)

# Convert the environment to a TFEnv to generate tensors.
tf_env = tf_py_environment.TFPyEnvironment(random_env)

Como hemos definido las observaciones como un diccionario, necesitamos crear capas de preprocesamiento para gestionarlas.

In [ ]:
preprocessing_layers = {
    'image': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
                                        tf.keras.layers.Flatten()]),
    'vector': tf.keras.layers.Dense(5)
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
actor = ActorNetwork(tf_env.observation_spec(), 
                     tf_env.action_spec(),
                     preprocessing_layers=preprocessing_layers,
                     preprocessing_combiner=preprocessing_combiner)

Ahora que ya tenemos la red de actores, podemos procesar observaciones del entorno.

In [ ]:
time_step = tf_env.reset()
actor(time_step.observation, time_step.step_type)

Esta misma estrategia se puede utilizar para personalizar cualquiera de las principales redes que utilizan los agentes. Puede definir cualquier preprocesamiento y conectarlo al resto de la red. A medida que define su propia personalización, asegúrese de que las definiciones de la capa de salida de la red coincidan.